In [49]:
import sqlite3
import plotly.express as px

from data import AlphaVantageApi, SQLRepository
from model import GarchModel
from config import settings
import pandas as pd
import plotly.graph_objects as go


In [50]:
def build_model(ticker, use_new_data=False):

    #Create DB connection
    connection = sqlite3.connect(settings.db_name, check_same_thread=False)

    #Instantiate the AlphaVantage class
    api = AlphaVantageApi()

    #Instantiate the repo class
    repo = SQLRepository(connection=connection)

    #Instantiate the model class
    model = GarchModel(ticker=ticker, repo=repo, use_new_data=use_new_data)

    return model

In [51]:

ticker = "IBM"
use_new_data = False
n_observations = 2000
p = 1
q = 1

    #Use try block to handle exception
try:
    #Build model with model_build function
    model = build_model(ticker, use_new_data)

        #Wrangle data
    model.wrangle_data(n_observations=n_observations)

        #Fit the model
    model.fit(p=p, q=q)

        #Save model
    filename = model.dump()

except Exception as e:
    print (e)

print(f"Model saved at {filename}")

Model saved at models/2023-03-10T12:14:42.488006_IBM.pkl


In [52]:
ticker = "IBM"
n_days = 5

try:
    #Build model with build_model function
    model = build_model(ticker = ticker, use_new_data = False)

    #Load stored model
    model.load()

    #Generate prediction
    prediction = model.predict_volatility(horizon = n_days)

except Exception as e:
        print(e)

print(prediction)

{'2023-03-10T00:00:00': 1.2886593054825333, '2023-03-13T00:00:00': 1.3109233262448257, '2023-03-14T00:00:00': 1.3308174243898248, '2023-03-15T00:00:00': 1.3486279514081354, '2023-03-16T00:00:00': 1.3645993418908986}


In [78]:
da_pred = pd.DataFrame(prediction, index=["prediction"]).transpose()
ds_pred = da_pred["prediction"]
ds_pred

2023-03-10T00:00:00    1.288659
2023-03-13T00:00:00    1.310923
2023-03-14T00:00:00    1.330817
2023-03-15T00:00:00    1.348628
2023-03-16T00:00:00    1.364599
Name: pred, dtype: float64

In [82]:
fig = px.line(ds_pred, x=ds_pred.index, y= "pred", markers=True, title = f"{ticker} 5 Day Volatility Prediction")
fig.update_layout(xaxis_title = "Date", yaxis_title = "Price")
fig.show()

In [54]:
def plot_graph(ticker, graph_type, n_observations = 2000):
    """ Plot Graph
    Parameter
    ========
    Input: ticker: str, graph_type: str

    Return
    ======
    Output: graph fig
    """
    try:
        #check if the table exists in the database
        connection = sqlite3.connect(settings.db_name, check_same_thread=False)
        repo = SQLRepository(connection=connection)
        res = connection.execute("SELECT name FROM sqlite_master WHERE name= '%s' " %ticker)
        if res.fetchone() != None:
            #if the table exist, check if the data is up to date
            data = repo.read_table(table_name=ticker, limit=2500+1)
            if data.index.max() == pd.to_datetime('today').normalize():
                data = data
            else:
                #Download a new data using the ticker
                api = AlphaVantageApi()
                data = api.get_historical_data(ticker=ticker)
                repo.insert_table(table_name = ticker, records = data, if_exists="replace")
                data = repo.read_table(table_name=ticker, limit=n_observations + 1)
        else:
             #Download a new data using the ticker
            api = AlphaVantageApi()
            data = api.get_historical_data(ticker=ticker)
            repo.insert_table(table_name = ticker, records = data, if_exists="replace")
            data = repo.read_table(table_name=ticker, limit=2500 + 1)

        data.sort_index(ascending = True, inplace = True)
        data["return"] = data["close"].pct_change() * 100
        data = data.dropna()
        #Build the graph
        #value = "Price"
        if graph_type == "Volatility":
            data['rolling_6d_volatility'] = data['return'].rolling(window=6).std().dropna()
            fig = px.line(data, x=data.index, y='rolling_6d_volatility', title = f"{ticker} 6D Rolling Volatility")
            fig.update_layout(xaxis_title = "Date", yaxis_title = "Price")
            
            
        else:
            fig = px.line(data, x=data.index, y='close', title = f"{ticker} Historical Price")
            fig = go.Figure(fig)
            fig.update_layout(xaxis_title = "Date", yaxis_title = "Price")
        

    except Exception as e:
        print(e)
        
    return fig

In [55]:
plot_graph(ticker="IBM", graph_type="Volatility")

In [56]:
if 2 == 4:
    print("correct")
    if 3 == 4:
        print("3")
    else:
        print (4)
else:
    print(5)

5


In [57]:
item_list = [
    "AGZ-iShares Agency Bond ETF ",
    "AMJ-JPMorgan Chase & Company ETF",
    "BAD-B.A.D. ETF",
    "CUT-Invesco MSCI Global Timber ETF",
    "ACB-Aurora Cannabis Stock",
    "IBM-International Business Machines Corp Stock",
    "ADBE-Adobe Stock",
    "ADP-Automatic Data Processing Inc Stock"
]

In [58]:
for item in item_list:
    print(item.split('-')[0])

AGZ
AMJ
BAD
CUT
ACB
IBM
ADBE
ADP


In [59]:
options=[{"label": x, "value": x} for x in sorted(item_list)]

In [60]:
list(options[0].values())[1]

'ACB-Aurora Cannabis Stock'

In [61]:
import os
file_paths = os.listdir("models/")

In [62]:
file_paths

['2023-01-09T22:19:30.497246_IBM.pkl',
 '2023-03-09T09:34:43.610409_IBM.pkl',
 '2023-03-10T12:14:42.488006_IBM.pkl']

In [63]:
fitted_ticker = file_paths[0].split('.')[1].split('_')[1]

In [48]:
fitted_date = file_paths[0].split('T')[0]

'2023-01-09'

In [64]:
today_date = pd.to_datetime('today').normalize()

In [84]:
today_date = pd.to_datetime('today').normalize()
for path in file_paths:
    fitted_ticker = file_paths[0].split('.')[1].split('_')[1]
    fitted_date = file_paths[0].split('T')[0]

    if (ticker == fitted_ticker) & (fitted_date == today_date):
        print("fitted model present")
        #call the prediction function
    break
print("No fitted model")

#call the fit function
#call the prediction function
#plot the predictions on a graph


No fitted model


In [85]:
from enginehouse import ProcessWorkflow

In [86]:
wk = ProcessWorkflow()

In [87]:
fig = wk.plot_graph("IBM", graph_type="Volatility")

In [88]:
filename = wk.fit_model(use_new_data=False, p=1, q=1)

In [89]:
print(filename)

models/2023-03-10T21:25:21.719935_IBM.pkl


In [90]:
prediction = wk.predict_volatility(n_days=5)

In [91]:
prediction

{'2023-03-10T00:00:00': 1.2886593054825333,
 '2023-03-13T00:00:00': 1.3109233262448257,
 '2023-03-14T00:00:00': 1.3308174243898248,
 '2023-03-15T00:00:00': 1.3486279514081354,
 '2023-03-16T00:00:00': 1.3645993418908986}

In [93]:
file_paths = os.listdir("models/")
file_paths

['2023-01-09T22:19:30.497246_IBM.pkl',
 '2023-03-10T22:43:31.619374_ACB.pkl',
 '2023-03-10T22:44:23.379969_ACB.pkl']

In [119]:
import datetime
ticker = "ACB"
file_paths = os.listdir("models/")
today_date = datetime.datetime.now().date().strftime('%Y-%m-%d')
for path in file_paths:
    fitted_ticker = path.split('.')[1].split('_')[1]
    #print(fitted_ticker)
    fitted_date = path.split('T')[0]
    #print(fitted_date)
    if  (ticker == fitted_ticker) & (fitted_date == today_date):
            #call the prediction function
        predict = True
    #call the fit function
    else:
        predict = False 
if predict:
    print("plot_graph") 
else:
    print("fit_model")   
    #call the prediction function

plot_graph


In [117]:
fitted_date 

'2023-03-10'

In [102]:
today_date = pd.to_datetime('today').normalize()
today_date

Timestamp('2023-03-10 00:00:00')

In [97]:
fitted_date == today_date

False

In [107]:
import datetime
datetime.datetime.now().date().strftime('%Y-%m-%d') == fitted_date

True